# HACKTM 2023
## kaitenzushi

Attachments:
```python
from math import gcd
from Crypto.Util.number import bytes_to_long, isPrime

from secret import p, q, x1, y1, x2, y2, e, flag

# properties of secret variables
assert isPrime(p) and p.bit_length() == 768
assert isPrime(q) and q.bit_length() == 768
assert isPrime(e) and e.bit_length() == 256
assert gcd((p - 1) * (q - 1), e) == 1
assert x1.bit_length() <= 768 and x2.bit_length() <= 768
assert y1.bit_length() <= 640 and y2.bit_length() <= 640
assert x1 ** 2 + e * y1 ** 2 == p * q
assert x2 ** 2 + e * y2 ** 2 == p * q

# encrypt flag by RSA, with xor
n = p * q
c = pow(bytes_to_long(flag) ^^ x1 ^^ y1 ^^ x2 ^^ y2, e, n)
print(f"{n = }")
print(f"{c = }")

# hints 🍣
F = RealField(1337)
x = vector(F, [x1, x2])
y = vector(F, [y1, y2])
# rotate
theta = F.random_element(min=-pi, max=pi)
R = matrix(F, [[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
x = R * x
y = R * y
print(f"{x = }")
print(f"{y = }")
```

output.txt:

In [37]:
n = 990853953648382437503731888872568785013804329239290721076418541795771569507440261620612308640652961121590348037236708702361580700250705591203587939980126323233833431892076634892318387020242015741789265095380967467201291693288654956012435416445991341222221539511583706970342630678909437274145759598920314784293470918464283814408418704426938549136143925649863711450268227592032494660523680280136089617838412326902639568680941504799777445608524961048789627301462833
c = 312168688094168684887530746663711142224819184527420449851136749248641895825646649162310024737395663075921549510262779965673286770730468773215063305158197748549937395602308558217528064655976647148323981103647078862713773074121667862786737690376212246588956833193632937835958166526006128435536115531865213269197137648990987207140262543956087199861542889002996727146832659889656384027201202873352819689303456895088190857667281342371263570535523695457095802010885279
x = ("9.93659400123277470926327676478883140697376509010297766512845139881487348637477791719517951397052010880811619509960535668814993293095146708957649613776125686226138447162258666762024346093786649228730054881453449071976210130217897905782845690384638460560301964009359233596889465133986468021963885911072779457835979983964294586954038412718305000570678333513135467257498071686562749882446495426493483289204e230", "-1.20540611958254673086539287012513674064476659427085664430224592760592531301348857885707154893714440960111029743010026152632150988429192286517249118913535366887447596463819555191858702861383725310592687577510708180057642425944345656558038998574368521689142109798891989865473206201635908814994474491537093810680632691594902962470061189337645818851446622588020765058461348047229165216450857822980873846637e230")
y = ("9.02899744041999015549480362358897037217795303901085937071039171882835297563545959015336648016772002396355451308252077767567617065937943765701645833054147976124287566465577491039263554806622908070370269238064956822205986576949383035741108310668397305286076364909407660314991847716094610949669608550117248147017329449889127749721988228613503029640191269319151291514601769696635252288607881829734506023770e191", "2.82245306887391321716872765000993510002376761684498801971981175059452895101888694909625866715259620501905532121092041448909218372087306882364769769589919830746245167403566884491547911250261820661981772195356239940907493773024918284094309809964348965190219508641693641202225028173892050377939993484981988687903270349415531065381420872722271855270893103191849754016799925873189392548972340802542077635974e192")

There are a lot of scary things going on here.
First of all we know that there're at least two positive solutions to the equation $x^2 + e y^2 = n$. However we have no idea what are $e, x_1, x_2, y_1, y_2$...

Yeah, so, let's go from the top.
We are given the rotation of two vectors $\vec{x} = (x_1, x_2), \vec{y}=(y_1, y_2)$, however it's done in RealField with precision of 1337 bits. Doing several calculations we know that we approximately have  

In [38]:
(log(2^1337)/log(10)).n()

402.477104202743

Decimal precision. My first idea was to find norms of these vectors. We can do that, since the rotation matrix has the determinant equal 1 hence the norm is not affected.

In [39]:
F = RealField(1337)
x1 = vector(F, x)
y1 = vector(F, y)

In [40]:
y1 * y1

8.78146927378687804346849516431701507826484739348080172078581295343203933796288398403780265407039536677360824145942988674652665107092911797701256927654801309745388007503323473078979690948112414174788639700290369327675404929545010203356471363412085360928525264568944158078505956293955516210257489570955889293358066585714260764574422111567556550925684212904129363328911799410215159707348799999999999999997e384

384 decimal points and bunch of 9999s at the end. Very cute

In [41]:
A = round(y1 * y1)
A

8781469273786878043468495164317015078264847393480801720785812953432039337962883984037802654070395366773608241459429886746526651070929117977012569276548013097453880075033234730789796909481124141747886397002903693276754049295450102033564713634120853609285252645689441580785059562939555162102574895709558892933580665857142607645744221115675565509256842129041293633289117994102151597073488

However x is not so nice

In [42]:
x1 * x1

1.00188904258462216509670716282096964126193651860611493228344983368984260549423216223342730814911020689877464832390119758498880054429876720749891748968776034889306616577033000366617106410031333685785841551551745932707057613281813684785116494745655476235178651477699642259165563504969297336293342132437464514606608479664028139182092203323073824300120945241571338185747828998166218935838012449899628955894e462

In [43]:
bin(round(x1 * x1))

'0b1101010011010010011001011101100110010110101000111101000010101100111011111010100100000111001001101111101010011010001100100110110100111011011010000110000010101111101111110100110110001001111011110010101011010100101010001110010110111111010101101100001111000111101001101010100110001110110010101010101111100111011110011001000100011111101110111101100010101111011010010000011110110100011001011110001111001000101100101111001010011100000010011011001100001001100100111111001001110011101010011110110110010110100001001100100110001101000101000010011100110010100010110011000001110001000111110000010111110101010001001011000000101110000000001001110010110111010110110010000001110011011000101000010010101010010011110111101001101011101111001000101110101100101001010101111001110011101000001101101001101000110110010101101000001011000010001100010101100100010100001001101010100101110111110010001000010100100001001110011111000000010100101101001000100011001100110001001111001110110011011011001011010100111100111011111111111

It's quite bigger than 402. As you can see there is a big party of zeros at the end of it's norm. Thanks to precision. 

We still can recover it using the nice fact that $(x_1^2 + x_2^2) + e (y_1^2 + y_2^2) = 2 n$ and taking the remainder modulo $A = (y_1^2 + y_2^2)$.

It's nice that the remaining part is strictly less than $\frac{A}{2}$

In [45]:
X = round(x1 * x1)

xn = (2 * n - X) % A

if xn > A//2:
    xn = xn - A
    
B = X + xn

e = (2 * n - B) // A
assert is_prime(e)
assert int(e).bit_length() == 256
assert B + A * e == 2 * n

Now let's look at this strange equation:

$(x_1 * cos(\theta) - x_2 * sin(\theta)) * (y_1 * sin(\theta) + y_2 * cos(\theta)) - (y_1 * cos(\theta) - y_2 * sin(\theta)) * (x_1 * sin(\theta) + x_2 * cos(\theta)) = x_1 * y_2 - x_2 * y_2$ 

It's still not quite good, due to precision, however we got some trics. But first: why should we care about this THING. Well, let's look at the equations above:

$x_1^2 + e y_1^2 = n$

$x_2^2 + e y_2^2 = n$

As we can see $\frac{x_i}{y_i}$ is the square root of $e$ modulo $n$. There were a lot of asserts at the beginning of the chall.py file. They literally mean that there're two distinct solutions to equations, hence we obtain two distinct square roots of $e$ modulo $n$. And that means that we are able to factor it out. Since $z_1^2 = z_2^2\ (mod\ n)$, therefore $(z_1 + z_2)(z_1 - z_2) = 0\ (mod\ n)$ They are distinct, hence $z_1 != z_2$ we have to admit that $z_1 + z_2$ and  $z_1 - z_2$ contain $p$ and $q$. Then we can take gcd to so on to so for. 

Moreover:

$\frac{x_1^2}{y_1^2} = \frac{x_2^2}{y_2^2}\ (mod\ n)$ => $x_1^2y_2^2 - x_2^2y_1^2 = (x_1y_2 - x_2y_1)(x_1y_2 + x_2y_1) = 0\ (mod\ n)$ That's why we are so interested in the sum above(and below now).

In [48]:
X = round(x1[0] * y1[1] - x1[1] * y1[0])
bin(X)

'0b1101001010010110101101111111001011001110010011101110010001100001001010011110111010011100011101001010010110101100010001011101111110101001011111011100111001010001110101111000101010000001110000001110010011001001010011000101001000011010011110011111001000111100010011111101000011101110011100111000011010101111100010010011101010101001110000111011100111001110100100001100001011001101101101101111000001001011010110010100001110000111001110011111110101100111010000001100111000000000110010100110101000100111010010010101100101010110000010011001110101100101000010101101001000110011111011110010111001001000111000111100001100000011000011110100000000011101100101110100101011011101001011101001000100101101111110011011100100011101001010100101111110001010110100001110001001011000001001001010001011110001111111110101001100101001001001001111111010110001111101101110011010001110011010111100101001100101110111110010100111011110011000000010010000111110101100011011100111111101001101000110011000001110010101011101000010011

We know that $x_1y_2 - x_2y_1$ is divisible by either p or q. And we can see that we miss at least 70 bits. Here comes Don Coppersmith. 

$x_1y_2 - x_2y_1 = X + x_n$, where $X$ we know and $x_n \approx 2^{73}$. We can use small roots of this equation modulo $n$ to find $x_n$.

In [49]:
r = var('r')
p = PolynomialRing(Zmod(n), r)(X + r).monic()
p.small_roots(X=2**73, beta=0.01)

[6595909465104215566284]

In [50]:
p = gcd(X + 6595909465104215566284, n)
q = n // p
assert p * q == n
p, q

(957509848415776008506125961998120495161250346184055094697245571121876444575553394581756735245207167681344755095903616730328731358607257251854603846193989936802222147961302618645021044609662945352893811478461448918625795339911124621,
 1034823772609519458248846092612745175801926111788817875968858172062391523082136646482602458010790281266293304602760763837135986847471585785747804498183711025365926891307517112408824936044882275438040847979334823200062286335134955573)

We successfully factored $n$, but we have no idea what $x_i, y_i$'s are. I have found the [Algorithm Revisited:
Solving at2 + btu + cu2 = n in the Case of Negative Discriminant](https://cs.uwaterloo.ca/journals/JIS/VOL17/Matthews/matt10.pdf). There's an algorithm at page 9, which helps us to find the integral points on our ellipse!. I won't get deep into details here. However you should notice, that there's a mistake in an algorithm: 

<b>if $aA^2_i + bA_iB_i + cB^2_i = 1$ then</b>

There should be 
```python

if P * Ai**2 + Q * Ai * Bi + R * Bi**2 == 1:
```

also I have omited several generalizations.

In [51]:
def get_frac(a, b):
    frac = []
    while a * b:
        c = a // b
        frac.append(c)
        a %= b
        b, a = a, b
    return frac


def get_partial_frac(r, n):
    x = r[n] + 1 / r[n + 1]
    for i in range(n - 1, -1, -1):
        x = r[i] + 1 / x
    return x


def calc_conv(q, n):                                   # calculate convergents of a fraction
    r = get_frac(q.numerator(), q.denominator())
    ans = get_partial_frac(r, n)
    return ans.numerator(), ans.denominator()


def solve_dioph(e, n, p, q):
    x = var("x")
    r1 = PolynomialRing(GF(p), x)(x**2 + e).roots()
    r2 = PolynomialRing(GF(q), x)(x**2 + e).roots()
    roots = []

    for i in r1:
        for j in r2:
            roots.append(crt([int(i[0]), int(j[0])], [p, q]))

    roots = [int(x) if int(x) < n // 2 else int(x) - n for x in roots]
    D = -4 * e

    solutions = []
    for teta in roots:
        assert (teta**2 + e) % n == 0
        
        P = (teta**2 + e) / n
        Q = -2 * teta
        R = n
        bound = ceil(sqrt(4 * P / (-D)))
        i = 0
        Ai, Bi = calc_conv(-Q / (2 * P), i)
        while Bi <= bound:
            if P * Ai**2 + Q * Ai * Bi + R * Bi**2 == 1:
                x, y = Ai * teta - n * Bi, Ai
                print(teta, x, y)
                solutions.append((x, y))
            i += 1
            Ai, Bi = calc_conv(-Q / (2 * P), i)
    return solutions

In [53]:
sols = solve_dioph(e = e, n = n, p = p, q = q)
sols

-329696169097498523396446198944496082037809518894188243766995172697455987801871682054561926788004773857296732077684784301372276119179972175242251981910556470481407188549665110813892388300986403675597304430267789920127527184710036003931967525382299448716923481827385031686386823126054656977832559278729157353715499278084301674325719597351630161041270649695020581775755564351276386888835317672063071293052857237467431998637220635116281788789348706467938215936000909 123343431936894440973263647479974540141395074556779828339916509613682879668610901423506961118285523166037774054833601787794419590891163752205158573276826154790166536984681500991748749778629881670438838666011425669518792357094873553 -2957028917590401838272414886210261099554152128524012256631787151968768935090286908219944634008304129914083074684507666539700290047827545862670465906725813971398170535104589598065683927537059268
-112295430716383809083039016922882355870464171020464913395282677788220894914370299313341321725029770903935696

[(123343431936894440973263647479974540141395074556779828339916509613682879668610901423506961118285523166037774054833601787794419590891163752205158573276826154790166536984681500991748749778629881670438838666011425669518792357094873553,
  -2957028917590401838272414886210261099554152128524012256631787151968768935090286908219944634008304129914083074684507666539700290047827545862670465906725813971398170535104589598065683927537059268),
 (993315378106395196440156892634615357425859001976376351903878161126954317590016249318316631584063366449446002974804447367756266228508159317926113473123770241598131922105753478630709094061327843793983555725542453353312556415777678937,
  -193518098174342694414424160720807163740044134017573004218248685165604434384710484681124817651698709818703976889508767807895216618103609127904817977547152172876909535027087606807328610207963608),
 (99331537810639519644015689263461535742585900197637635190387816112695431759001624931831663158406336644944600297480444736775626622850

The last step is to take all the absolute values of found solutions and decrypt ciphertext. 

In [54]:
s = set()
for x, y in sols:
    s.add((abs(x), abs(y)))
s = list(s)
x1, y1 = s[0]
x2, y2 = s[1]

In [55]:
d = pow(e, -1, (p - 1)  * ( q - 1))
m = pow(c, d, n)
m = int(m) ^^ int(x1) ^^ int(x2) ^^ int(y1) ^^ int(y2)
from Crypto.Util.number import long_to_bytes
print(long_to_bytes(int(m)))

b'HackTM{r07473_pr353rv35_50m37h1n6}'
